In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
pd.set_option('display.max_columns', None)

In [2]:
dataPath = "data/"

pairs = pd.read_csv(dataPath + "relation.csv")
users = pd.read_csv(dataPath + "customers.csv", index_col="id")
items = pd.read_csv(dataPath + "products.csv", index_col="id")


/tmp/ipykernel_996/1774659839.py:4: DtypeWarning: Columns (7,9,10,11,12,13,17,18,21,26,27,28,30,36) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv(dataPath + "customers.csv", index_col="id")
/tmp/ipykernel_996/1774659839.py:5: DtypeWarning: Columns (20,32,34,39) have mixed types. Specify dtype option on import or set low_memory=False.
  items = pd.read_csv(dataPath + "products.csv", index_col="id")


### Compañia 15

In [3]:
df_pairs = pairs[ pairs.company_id == 15]#.copy()
df_users = users[ users.company_id == 15]#.copy()
df_items = items[ items.company_id == 15]#.copy()

In [4]:
# Comprobacion de que no haya usuarios ni productos duplicados

assert df_users.index.duplicated().any()==False
assert df_items.index.duplicated().any()==False

### Quitar usuarios que interaccionan pero no estan contemplados en la tabla usuarios (unos 575)

In [5]:
usuarios_que_interaccionan_pero_no_en_tabla_usuarios = set(df_pairs.customer_id) - set(df_users.index)
len(usuarios_que_interaccionan_pero_no_en_tabla_usuarios)

575

In [6]:
df_pairs = df_pairs[ ~df_pairs.customer_id.isin(usuarios_que_interaccionan_pero_no_en_tabla_usuarios) ]
len(df_pairs)

59167

### Quitar productos que interaccionan pero no estan contemplados en la tabla productos (unos 162)

In [7]:
productos_que_interaccionan_pero_no_en_tabla_productos = set(df_pairs.product_id) - set(df_items.index)
len(productos_que_interaccionan_pero_no_en_tabla_productos)

162

In [8]:
df_pairs = df_pairs[ ~df_pairs.product_id.isin(productos_que_interaccionan_pero_no_en_tabla_productos) ]
len(df_pairs)

58913

# <center> Fin código limepieza datos
---
# <center> Inicio código Sistema de recomendación

In [9]:
from recommenderSystem2 import RecSys2
r = RecSys2()

In [10]:
r.entrenar_modelo(df_pairs, df_users, df_items)

Leyendo datos...
Entrenando modelo...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 921/921 [00:15<00:00, 57.91it/s]


tensor(3.7480, grad_fn=<MseLossBackward0>)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 921/921 [00:15<00:00, 58.05it/s]

tensor(1.8869, grad_fn=<MseLossBackward0>)
Generando embeddings...
Terminado


In [11]:
r.activeUser_ids[:5], r.newUser_ids[:5]

(tensor([21724, 21726, 21728, 21730, 21731]),
 tensor([21725, 21727, 21729, 21742, 21744]))

In [12]:
r.activeItem_ids[:5], r.newItem_ids[:5]

(tensor([28863, 28864, 28865, 28866, 28867]),
 tensor([28871, 28873, 28879, 28881, 28936]))

In [24]:
r.recomendar(query_user_ids=[21724, 21726], query_item_ids=[28863, 28864, 28865], peso_query_user=0.3,
            searchOn_act_users=True, limit=0.65)

torch.Size([2, 32]) torch.Size([32])
torch.Size([3, 32]) torch.Size([32])


tensor([167395,  25364,  30073,  44010,  30052,  33696,  52051,  51995, 161419,
        140554,  28960,  48820, 143774,  32085,  46274,  36079,  54672, 280950,
         40124,  47328,  32686,  32998, 213787,  31184,  56686, 144796,  31890,
         27853,  45491,  25624,  51216, 153966,  51141,  43774,  35872,  31451,
         23094, 165342,  53345, 505965,  58994,  33540, 529624,  56809,  29070,
         46701,  42442,  37552])

In [13]:
r.recomendar(query_user_ids=[21724, 21726], searchOn_act_users=True, limit=0.8)

tensor([170430,  53313,  21726,  21724, 146186, 518928, 231517, 231566,  35532,
        483088, 155420, 157056, 149349, 231531, 151480, 171887, 374717, 517240,
        529719, 140523,  42697, 533962,  78717, 140806, 230110, 170642, 537502,
        149114, 220129, 535114, 456423, 155714, 149462,  28387, 148240, 145249,
        141620, 165493, 146426, 518563, 512614, 148432, 151425, 307091,  24389,
         53913, 533417, 165905, 532995, 348414, 409654, 140676, 169043, 537517,
        461714, 536699, 140281, 172200, 159292, 231585, 533437,  40886,  23552,
        146062,  45457, 160643,  59396, 536282,  54547, 140340, 167863, 537411,
        536444, 429239,  56244, 149557,  73665,  37637,  35984, 151105, 530230,
         52462, 531640, 153479, 531012, 532140,  52028, 147025,  53932, 141590,
        535335, 142873, 165280, 532503, 170484, 148959,  44352, 536662, 517261,
        151334,  51583,  51148, 486566, 172246,  52959, 165230, 172202, 156747,
        506218, 142167,  27413,  72237, 

In [14]:
r.recomendar(query_item_ids=[28863, 28871], searchOn_act_users=True, limit=0.7)

tensor([534062, 230194, 121961,  50447,  51045,  73344,  29971,  39161,  43962,
        144200, 151248,  51932,  42768,  98707,  55195,  45013,  22973,  39304,
        170151,  44992, 123216, 156356,  49156, 145467,  52262,  39773,  57576,
         54834,  34503,  23073,  58964,  30387, 123164,  50791, 149566,  48407,
        140432, 506318,  53445, 474843, 195603, 230647, 122071,  50937,  59612,
         29070,  52587, 517911,  44589,  37460,  53749,  58572, 145251,  23097,
         24259,  53841,  59540,  53304,  51448, 170308, 163279,  40549, 230955,
         35714, 147995,  52028, 142936,  37440,  34545,  51143,  30446,  44456,
         53034,  76955, 477914,  40405, 536248, 123234, 531640, 140768,  73411,
         76789,  27466, 167827, 534103,  58618,  45690,  75322,  23753,  27076,
        123938,  58317, 195592,  24165,  55811, 290969,  46078,  60268,  48254,
        505717,  40628, 151205,  30452,  47764, 149505, 100339,  27973, 529671,
        141647,  48921,  26579,  36170, 

In [17]:
r.newUser_embs

tensor([[ 0.3719, -0.0594, -0.3835,  ...,  0.0198,  0.1380,  0.0437],
        [ 0.3719, -0.0594, -0.3835,  ...,  0.0198,  0.1380,  0.0437],
        [ 0.3719, -0.0594, -0.3835,  ...,  0.0198,  0.1380,  0.0437],
        ...,
        [ 0.3719, -0.0594, -0.3835,  ...,  0.0198,  0.1380,  0.0437],
        [ 0.3210, -0.1262, -0.2409,  ..., -0.0244,  0.1400,  0.1837],
        [ 0.3405, -0.0418, -0.3320,  ..., -0.0201,  0.0929,  0.2228]])

In [18]:
r.activeUser_embs

tensor([[ 0.3192, -0.0241, -0.2915,  ..., -0.1165,  0.1095,  0.0411],
        [ 0.3002, -0.0479,  0.0032,  ...,  0.0162,  0.1553, -0.0342],
        [ 0.4004, -0.0542, -0.3557,  ...,  0.0184, -0.0594,  0.2058],
        ...,
        [ 0.2329, -0.1110, -0.2166,  ..., -0.0215,  0.4049,  0.1638],
        [ 0.3030, -0.2221, -0.2273,  ..., -0.0230,  0.1322,  0.1733],
        [ 0.2253, -0.0509, -0.3024,  ...,  0.0155, -0.0522,  0.0342]])

In [19]:
r.activeItem_embs

tensor([[ 0.2910, -0.0460, -0.2071,  ...,  0.0156,  0.7747,  0.0952],
        [ 0.2818, -0.0875, -0.2786,  ..., -0.1080,  0.1367,  0.4187],
        [ 0.2780, -0.0760, -0.2691,  ...,  0.0087,  0.1218,  0.1433],
        ...,
        [ 0.3849, -0.0944, -0.1464,  ...,  0.0619,  0.1363,  0.2019],
        [ 0.0826, -0.0628, -0.2057,  ...,  0.0445,  0.0963,  0.0569],
        [ 0.1099,  0.1584, -0.2032,  ...,  0.0713,  0.0286, -0.1603]])